<a href="https://colab.research.google.com/github/Chetan-kr/Data-Engineering/blob/main/ETL_process%20for%20different%20file%20formats%20from%20zippedfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ETL**

Let's Extract--> Transform--> Load the data


In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime
import pytz                         # this module helps to get Asia/calcutta timezone

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2023-05-12 09:34:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2023-05-12 09:34:35 (359 MB/s) - ‘source.zip’ saved [2707/2707]



In [3]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


In [4]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored'

In [23]:
# csv extract function

def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [24]:
# json extract function

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [25]:
# xml extract function

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

In [26]:
# Extract function
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data


**Transform**


The transform function does the following tasks.

Convert height which is in inches to millimeter

Convert weight which is in pounds to kilograms

In [27]:
def transform(data):

  #Convert the datatype of the column into float
  data['height'] = data.height.astype('float')  
  #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
  data['height'] = round(data.height * 0.0254, 2)

  #Convert the datatype of the column into float
  data.weight = data.weight.astype(float)
  #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
  data['weight'] = round(data.weight * 0.45359237,2)
  
  return data



**Let's Load the Data into the Targetfile**

In [28]:
def load(targetfile, data_to_load):
  data_to_load.to_csv(targetfile)

**Let's Logging**

In [40]:
import pytz
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now(pytz.timezone('Asia/calcutta')) # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

**Let's start all ETL Processes**

In [41]:
log('ETL job Started')

In [42]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

<ipython-input-26-8c7ff8493267>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
<ipython-input-26-8c7ff8493267>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
<ipython-input-26-8c7ff8493267>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
<ipython-input-26-8c7ff8493267>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile),

,name,height,weight,Unnamed: 0
0,alex,1.67,51.25,0.0
1,ajay,1.82,61.91,1.0
2,alice,1.76,69.41,2.0
3,ravi,1.73,64.56,3.0
4,joe,1.72,65.45,4.0
...,...,...,...,...
73,ivan,67.62,114.14,NaN
74,simon,67.90,112.37,NaN
75,jacob,66.78,120.67,NaN
76,cindy,66.49,127.45,NaN


In [43]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight,Unnamed: 0
0,alex,0.04,23.25,0.0
1,ajay,0.05,28.08,1.0
2,alice,0.04,31.48,2.0
3,ravi,0.04,29.28,3.0
4,joe,0.04,29.69,4.0
...,...,...,...,...
73,ivan,1.72,51.77,NaN
74,simon,1.72,50.97,NaN
75,jacob,1.70,54.73,NaN
76,cindy,1.69,57.81,NaN


In [44]:
log('Load phase Started')
load(targetfile,transformed_data)
log("Load phase Ended")

In [45]:
log("ETL Job Ended")